In [83]:
import numpy as np
import pandas as pd

In [6]:
PLATES = ['Beer', 'Whiskey', 'Vodka', 'Tequila', 'Gin', 'Rum' ]

user_likes = np.array([
    [5,0,3,5,0,1],
    [0,5,5,0,0,0],
    [0,0,0,0,0,5],
    [3,2,2,0,5,1],
    [3,1,0,0,0,0],
]) 

In [8]:
user_likes.shape

(5, 6)

In [96]:
FEATURES = 15

def build_initial_factors(m):
    rows, columns = m.shape
    f1 = np.random.rand(rows, FEATURES)
    f2 = np.random.rand(columns, FEATURES)
    return f1,f2

def factorize(m, f1, f2, steps = 10000, alpha=0.02, beta=0.02, error_trigger=0.001):
    f2 = f2.T
    rows, columns = m.shape 
    last_error = 0
    for step in range(steps):
        for i in range(rows):
            for j in range(columns):
                if m[i][j] > 0:
                    eij = m[i][j] - np.dot(f1[i,:],f2[:,j])
                    for k in range(FEATURES):
                        f1[i][k] = f1[i][k] + alpha * (2 * eij * f2[k][j] - beta * f1[i][k])
                        f2[k][j] = f2[k][j] + alpha * (2 * eij * f1[i][k] - beta * f2[k][j])
        e = 0
        for i in range(rows):
            for j in range(columns):
                if m[i][j] > 0:
                    e = e + pow(m[i][j] - np.dot(f1[i,:],f2[:,j]), 2)
                    for k in range(FEATURES):
                        e = e + (beta/2) * (pow(f1[i][k],2) + pow(f2[k][j],2))
        last_error = e
        if e < error_trigger:
            print(f'>> Achieved an error less than {error_trigger} in {step} calculation steps')
            break
    print(f'FINISH: Error = {last_error}')
    return f1, f2.T 
    

def extrapolate(m):
    f1,f2 = build_initial_factors(m)
    f1n, f2n = factorize(m, f1, f2)
    return np.round(np.matmul(f1n, f2n.T), 2)

In [97]:
extrapolated_user_likes = extrapolate(user_likes)

FINISH: Error = 1.1807111570311628


In [100]:
likes_df = pd.DataFrame(user_likes, columns = PLATES)
extrapolated_likes_df = pd.DataFrame(extrapolated_user_likes, columns = PLATES)

In [101]:
likes_df

,Beer,Whiskey,Vodka,Tequila,Gin,Rum
0,5,0,3,5,0,1
1,0,5,5,0,0,0
2,0,0,0,0,0,5
3,3,2,2,0,5,1
4,3,1,0,0,0,0


In [102]:
extrapolated_likes_df

,Beer,Whiskey,Vodka,Tequila,Gin,Rum
0,4.98,2.66,2.99,4.99,6.02,1.00
1,4.86,4.98,4.98,5.26,7.39,1.96
2,1.85,2.11,1.96,2.00,3.86,4.98
3,3.00,1.99,2.00,3.01,4.98,1.00
4,2.99,1.00,1.29,2.90,3.23,0.38
